Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.

# Accessing a Bundle Workflow in Python

In this guide, we'll explore how to access a bundle in Python and use it in your own application. We'll cover the following topics:

1. **Downloading the Bundle**: First, you'll need to download the bundle from its source. This can be done using the `download` API.

2. **Creating a `BundleWorkflow`**: Once you have the bundle, you can create a `BundleWorkflow` object by passing the path to the bundle file as an argument to `create_worflow`.

3. **Getting Properties from the Bundle**: You can then retrieve the properties of the bundle by directly accessing them. For example, to get the version of the bundle, you can use `workflow.version`.

4. **Updating Properties**: If you need to update any of the properties, you can do so by directly overwriting them. For example, to update the max epochs of the bundle, you can use `workflow.max_epochs = 10`.

5. **Using Components in Your Own Pipeline**: Finally, you can use the components from the bundle in your own pipeline by accessing them through the `BundleWorkflow` object.


## Setup environment

In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[gdown, nibabel, tqdm, ignite]"

## Setup imports

In [6]:
import os
import tempfile
from pathlib import Path
from monai.config import print_config
from monai.bundle import download, create_workflow

print_config()

MONAI version: 1.2.0+103.ga7a690ff
Numpy version: 1.22.2
Pytorch version: 1.13.1+cu117
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: a7a690ffb781c806b73500e68ba05be6fcdfd11a
MONAI __file__: /workspace/Code/MONAI/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.11
ITK version: 5.3.0
Nibabel version: 5.1.0
scikit-image version: 0.21.0
scipy version: 1.10.1
Pillow version: 9.2.0
Tensorboard version: 2.9.0
gdown version: 4.7.1
TorchVision version: 0.14.1+cu117
tqdm version: 4.65.0
lmdb version: 1.4.1
psutil version: 5.9.4
pandas version: 1.5.2
einops version: 0.6.1
transformers version: 4.21.3
mlflow version: 2.4.0
pynrrd version: 1.0.0
clearml version: 1.11.1rc1

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



## Setup data directory

You can specify a directory with the `MONAI_DATA_DIRECTORY` environment variable.  
This allows you to save results and reuse downloads.  
If not specified a temporary directory will be used.

In [2]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

/workspace/Data


## Downloading the Bundle

In [3]:
download(name="spleen_ct_segmentation", bundle_dir=root_dir)

name spleen_ct_segmentation
version None
bundle_dir /workspace/Data
source github
repo None
url None
remove_prefix monai_
progress True
2023-09-01 08:36:39,219 - INFO - --- input summary of monai.bundle.scripts.download ---
2023-09-01 08:36:39,220 - INFO - > name: 'spleen_ct_segmentation'
2023-09-01 08:36:39,220 - INFO - > bundle_dir: '/workspace/Data'
2023-09-01 08:36:39,221 - INFO - > source: 'github'
2023-09-01 08:36:39,221 - INFO - > remove_prefix: 'monai_'
2023-09-01 08:36:39,222 - INFO - > progress: True
2023-09-01 08:36:39,223 - INFO - ---




2023-09-01 08:36:40,143 - INFO - Expected md5 is None, skip md5 check for file /workspace/Data/spleen_ct_segmentation_v0.5.3.zip.
2023-09-01 08:36:40,143 - INFO - File exists: /workspace/Data/spleen_ct_segmentation_v0.5.3.zip, skipped downloading.
2023-09-01 08:36:40,144 - INFO - Writing into directory: /workspace/Data.


## Creating a `BundleWorkflow`
Here we use `create_workflow` to create and initialize a `BundleWorkflow`. `create_workflow` can support both creating a config-based bundle by passing the config files and a python-based bundle by passing the bundle workflow name which should be subclass of `BundleWorkflow` and be available to import.

In [4]:
config_file = Path(root_dir) / "spleen_ct_segmentation" / "configs" / "train.json"

train_workflow = create_workflow(config_file=config_file, workflow_type="train")

workflow_name None
config_file /workspace/Data/spleen_ct_segmentation/configs/train.json
workflow_type train
2023-09-01 08:36:40,372 - INFO - --- input summary of monai.bundle.scripts.run ---
2023-09-01 08:36:40,373 - INFO - > config_file: PosixPath('/workspace/Data/spleen_ct_segmentation/configs/train.json')
2023-09-01 08:36:40,374 - INFO - > workflow_type: 'train'
2023-09-01 08:36:40,375 - INFO - ---


2023-09-01 08:36:40,376 - INFO - Setting logging properties based on config: /workspace/Data/spleen_ct_segmentation/configs/logging.conf.


## Getting Properties from the Bundle
You need to check all supported properties listed [here](https://docs.monai.io/en/latest/mb_properties.html).

You can also use the `add_property` method of the `BundleWorkflow` object to add the property for the application requirements check and access.

In [5]:
# for existing properties
# Note that the properties got from `train_workflow` is already instantiated.
train_preprocessing = train_workflow.train_preprocessing

# for meta information
version = train_workflow.version

# add properties
train_workflow.add_property(name="lr_scheduler", required=True, config_id="lr_scheduler")
print(train_workflow.lr_scheduler)

monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.


## Updating Properties
There are two ways to update the properties:
- You can override them when you create the workflow
- Or you can directly overwriting them after creating the workflow 

In [7]:
# 1 override them when you create the workflow
dataset_dir = Path(root_dir) / "Task09_Spleen"
override = {"max_epochs": 3, "dataset_dir": dataset_dir}
train_workflow = create_workflow(config_file=config_file, workflow_type="train", **override)

workflow_name None
config_file /workspace/Data/spleen_ct_segmentation/configs/train.json
workflow_type train
max_epochs 3
dataset_dir /workspace/Data/Task09_Spleen
2023-09-01 08:42:38,109 - INFO - --- input summary of monai.bundle.scripts.run ---
2023-09-01 08:42:38,110 - INFO - > config_file: PosixPath('/workspace/Data/spleen_ct_segmentation/configs/train.json')
2023-09-01 08:42:38,111 - INFO - > workflow_type: 'train'
2023-09-01 08:42:38,111 - INFO - > max_epochs: 3
2023-09-01 08:42:38,111 - INFO - > dataset_dir: PosixPath('/workspace/Data/Task09_Spleen')
2023-09-01 08:42:38,111 - INFO - ---


2023-09-01 08:42:38,112 - INFO - Setting logging properties based on config: /workspace/Data/spleen_ct_segmentation/configs/logging.conf.
